## **VALIDACION CRUZADA Y ALEATORIDAD INICIAL**

---



### **<font color=red> LA INFLUENCIA DE LA ALEOTORIDAD EN LA EVALUACION DE UN MODELO**

In [ ]:
# 1°) Vemos la base de datos con la que vamos a trabajar, podemos ver que es una base de datos separada por comas, donde tenemos
# diferentes columnas, la besa de datos hace refernecia a un conjuntos de autos en los cuales algunos fueron vendidos y otros No --->

import pandas as pd

uri = "https://gist.githubusercontent.com/jessicadesousa/56ac5efd1b9f2a06ef503485e904a203/raw/d829a7204ad55506644f4a7058090c84424217e7/machine-learning-carros-simulacion.csv"
datos = pd.read_csv(uri)
datos.head()

,precio,vendido,edad_del_modelo,km_por_ano
0,30941.02,1,18,35085.22134
1,40557.96,1,20,12622.05362
2,89627.50,0,12,11440.79806
3,95276.14,0,3,43167.32682
4,117384.68,1,4,12770.11290


In [ ]:
# 2°) Como EVALUAMOS y ENTRENAMOS un modelo, tenemos que separar nuestra base de datos en VARIBLES EXPLICATIVAS(x = precio, edad del modelo,
# km por año) y lo que queremos hacer es cual de esos autos tiene mas probabilidad de ser vendido y cuales NO, tambien agregamos la VARIABLE
# RESPUESTA(y = vendido). Reservamos el 25% para EVALUACION(test_size=0.25) --->

import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score

x = datos[["precio", "edad_del_modelo", "km_por_ano"]]
y = datos["vendido"]

SEED = 158020
np.random.seed(SEED)

train_x, test_x, train_y, test_y = train_test_split(x, y,
                                                    test_size = 0.25,
                                                    stratify = y)

print("Entrenaremos con %d elementos y evaluaremos con %d elementos" %(len(train_x), len(test_y)))

Entrenaremos con 7500 elementos y evaluaremos con 2500 elementos


In [ ]:
# 3°)Cual seria mi base de comparacion?, Que es lo que yo deberia estar esperando para ese Modelo?
# En un caso real cuando estamo trabajando con una base de datos de una empresa preguntamos cual es el porcentaje de exito de ventas
# pero en eset caso NO tenemos a quien preguntar de la empresa cual es el ideal en ventas, podemos entoneces hacer un DUMMY CLASSIFIER
# el va a crear en cada linea una probabilidad de ser vendido o NO --->

from sklearn.dummy import DummyClassifier

dummy_stratified = DummyClassifier(strategy="stratified")
dummy_stratified.fit(train_x, train_y)
accuracy = dummy_stratified.score(test_x, test_y) * 100

print("La exactitud(accuracy) del dummy stratified fue %.2f%%" % accuracy)

# La exactitud(accuracy) del dummy stratified fue 50.96%, USAMOS ESTA MEDIDA PARA VER QUE TAN BIEN LE VA A NUESTRO MODELO.

La exactitud(accuracy) del dummy stratified fue 50.96%


In [ ]:
# 4°) Podemos crear un modelo de DECISION TREE de Clasificacion, lo creamos con solo 2 de profundidad(max_depth=2) y luego
# vamos a ENTRENAR y EVALUAR como se comporta en nuestra base de Evaluacion --->

from sklearn.tree import DecisionTreeClassifier

SEED = 158020
np.random.seed(SEED)

modelo = DecisionTreeClassifier(max_depth=2)
modelo.fit(train_x, train_y)

predicciones = modelo.predict(test_x)

accuracy = accuracy_score(test_y, predicciones) * 100
print("La exactitud(accuracy) fue %.2f%%" % accuracy)

La exactitud(accuracy) fue 71.92%


In [ ]:
# 5°) Ahora supongamos que nuestro grupo de trabajo de la empresa quiere que un ACCURACY del 73%, para esto vamos a ENTRENAR y EVALUAR
# el modelo nuevamente cambiando el SEED que es el que decide cuales datos van para ENTRENAMIENTO y cuales para EVALUCION --->

SEED = 5
np.random.seed(SEED)

train_x, test_x, train_y, test_y = train_test_split(x, y,
                                                    test_size = 0.25,
                                                    stratify = y)

modelo = DecisionTreeClassifier(max_depth=2)
modelo.fit(train_x, train_y)

predicciones = modelo.predict(test_x)

accuracy = accuracy_score(test_y, predicciones) * 100
print("La exactitud(accuracy) fue %.2f%%" % accuracy)

La exactitud(accuracy) fue 76.84%


### **<font color=red> VALIDACION CRUZADA**


Ejemplo:

Aprendimos sobre validación cruzada, cómo funciona, cómo la utilizamos y ventajas de usarla. Entonces, considerando que tenemos 12.
Sabiendo que la validación cruzada tiene una parte de entrenamiento y una de evaluación, ¿con cuántos elementos podemos entrenar?

* Podemos entrenar con diferentes N elementos.
Podemos entrenar con numerosos elementos, pues la validación cruzada nos permite esto.

### **<font color=red> USANDO Y EVALUANDO CON CROSS VALIDATE**


In [ ]:
# 1°) Vemos como EVALUAR con CROSS_VALIDATION, usaremos la funcion CROSS_VALIDATE y le pasaremos un Estimador que seria nuestro MODELO,
# nuestros datos para que el aprenda y entrene varias veces --->

from sklearn.model_selection import cross_validate

modelo = DecisionTreeClassifier(max_depth=3) # Definimos el Modelo que vamos a utilizar

resultados = cross_validate(modelo, x, y, cv = 3)
# X, Y = Pasamos solo x, y para que Cross Validation decida quien sera nuestra base de TRAIN y TEST.
# CV = 3, cuantas veces va hacer cross_validate, entra y evalua el modelo en 3

resultados

# TEST_SCORE, metrica que esta performando en nuestra base de TEST.

{'fit_time': array([0.01768732, 0.02016068, 0.02101636]),
 'score_time': array([0.00591969, 0.00539517, 0.00535583]),
 'test_score': array([0.78554289, 0.79057906, 0.78487849])}

In [ ]:
# Imprimimos TEST_SCORE, nos interesa solo estos valores:

resultados["test_score"] # Nos devuelve un array de 3 valores que fueron los cv=3

array([0.78554289, 0.79057906, 0.78487849])

In [ ]:
# 2°) Calculamos la MEDIA para obtener luego el INTERVALO de CONFIANZA, ya que la media no es suficiente , ya que nos
# da valores puntuales a diferencia del Intervalo de Confianza --->

media = resultados["test_score"].mean()
media

# Nos devuelve la media de esos 3 partes del cv=3

0.7870001457254303

In [ ]:
# 3°) Ahora obtenemos el INTERVALO de CONFIANZA para ver como esta varianado nuestra Metrica, SI asumimos que esas metricas tienen
# una distribucion Normal, como seria en este caso, podemos asumir que si usamos 2 Desviaciones Estandar alrededor de la MEDIA podemos
# tener un INTERVALO de CONFIANZA del 95% --->

desviacion_standard = resultados["test_score"].std()
print("Nuestro Intervalo de Confianza para CV = 3 es [%.2f, %.2f]" % ((media - 2*desviacion_standard)*100, (media + 2*desviacion_standard)*100))

# Nuestro INTERVALO de CONFIANZA para CV=3 es de [78.19, 79.21]

Nuestro Intervalo de Confianza para CV = 3 es [78.19, 79.21]


In [ ]:
# Si cambiaramos el SEED de 5 a 234, vemos que nuestra Metrica NO esta cambiando --->

SEED = 234
np.random.seed(SEED)

modelo = DecisionTreeClassifier(max_depth=3)
resultados = cross_validate(modelo, x, y, cv = 3)
resultados["test_score"]

media = resultados["test_score"].mean()
media

desviacion_standard = resultados["test_score"].std()
print("Nuestro Intervalo de Confianza para CV = 3 es [%.2f, %.2f]" % ((media - 2*desviacion_standard)*100, (media + 2*desviacion_standard)*100))

# Como CROSS_VALIDATE es determinante no importa que SEED le asignemos!

Nuestro Intervalo de Confianza para CV = 3 es [78.19, 79.21]


In [ ]:
# 4°) Ahora vamos a cambiar nuestro CV = 5 --->

SEED = 234
np.random.seed(SEED)

modelo = DecisionTreeClassifier(max_depth=3)
resultados = cross_validate(modelo, x, y, cv = 5)
resultados["test_score"]

media = resultados["test_score"].mean()
media

desviacion_standard = resultados["test_score"].std()
print("Nuestro Intervalo de Confianza para CV = 5 es [%.2f, %.2f]" % ((media - 2*desviacion_standard)*100, (media + 2*desviacion_standard)*100))

Nuestro Intervalo de Confianza para CV = 5 es [77.78, 79.60]


In [ ]:
# 5°) Vemos con CV = 10 --->

SEED = 234
np.random.seed(SEED)

modelo = DecisionTreeClassifier(max_depth=3)
resultados = cross_validate(modelo, x, y, cv = 10)
resultados["test_score"]

media = resultados["test_score"].mean()
media

desviacion_standard = resultados["test_score"].std()
print("Nuestro Intervalo de Confianza para CV = 10 es [%.2f, %.2f]" % ((media - 2*desviacion_standard)*100, (media + 2*desviacion_standard)*100))

Nuestro Intervalo de Confianza para CV = 10 es [77.15, 80.23]


* <font color=orange> Lo ideal es hacer CROSS_VALIDATE entre 5 y 10, segun documentacion de la funcion, para poder considerar nuestro INTERVALO de COFIANZA estable.

## **K-FOLD CON ALEATORIEDAD**

---


### **<font color=red> K-FOLD**

Vimos cómo calcular nuestro intervalo de confianza para cross-validation variando diferentes valores de este cv en cross-validation. Por eso nos dimos cuenta cambiando este SEED no nos estaba afectando mucho nuestro resultado en nuestro invervalo de confianza.

Entonces por ejemplo con este 3, 5, 7 con seed nos da este intervalo de confianza de 74.24 hasta 77.32, y si yo cambio ahorita para 1, 2, 3 por ejemplo me continúa dando 74.24 a 77.32. Entonces ¿qué es lo que está pasando? Parece ser que ese cross-validate de hecho no está siendo afectado por nuestro seed.

Vamos a ver aquí en la documentación para ver un poco qué es lo que está pasando. Vamos a aumentar un poco la fuente. Y si nos damos cuenta cross-validate no depende de ningún parámetro aleatorio que está aquí. Entonces si vemos, él no recibe ningún seed, ningún random-state que es común cuando uno quiere hacer nuestros splits determinísticos.

En realidad lo que está pasando es que este CV que está aquí dependiendo de cuál es el parámetro que él recibe, ya él es determinístico. Entonces, por ejemplo, vamos a ver aquí en la documentación y vemos que este cv puede ser un entero, que es lo que nosotros le pasamos. Nosotros le dijimos: "Yo quiero 10 splits o 5 splits".

O él puede ser un generador de cross-validation. Entonces en nuestro caso estamos usando en el ejemplo que acabamos de rodar usamos un cv igual a 10 y él lo que está usando en nuestro caso es un stratifier K-fold. Entonces en este caso centramos aquí en la documentación de stratifier K-fold, nos damos cuenta que él recibe un número de splits, que es lo que le acabamos de pasar allá, da ese cv.

Y él recibe un parámetro de SHUFFLE, que él nos dice si queremos desordenar los datos antes de comenzar a hacer los splits que él ya va a hacer. Entonces en nuestro caso como no le pasamos ningún shuffle, él de hecho mantiene de cierta manera el orden de los datos que les estoy pasando.

Entonces como no le pasé ningún parámetro diciéndole: "Bueno, desordéname los datos antes de explicar lo que está pasando", es que él mantiene en esos 5 splits el orden de alguna forma de los datos que le estoy pasando, entonces no importa si le digo: "Bueno, mi seed ahora es 5, después es 7, después puede ser 10".

En realidad ese parámetro no entra en ningún lugar porque sin ese shuffle, mi stratifier K-fold en este caso el que está utilizando aquí, es totalmente DETERMINISTICO. Entonces, él ya sabe cuáles son las líneas que va a usar porque divide en cinco partes y él ya sabe cuáles serían las primeras cinco partes, las segundas cinco partes y así.

En realidad no quiero que no me desordene los datos. QUIERO QUE TENGA UNA PARTE DESORDENADA. Entonces, lo que vamos a hacer en la clase de hoy es exactamente pasarle, en vez de decirle cuántos quiero, en este cv, en vez de decirle: "Quiero cinco splits", le vamos a pasar ese cross-validation generator. Le vamos a pasar un generador de splits.

Entonces en nuestro caso vamos a usar un K-fold hoy, que vamos a verlo aquí qué es lo que necesito, entonces ese K-fold lo que va a recibir es cuántos splits quiero y va a decir "bueno, desordéname o no me desordenes los datos antes de hacer ese split".


In [ ]:
# 1°) Lo que queremos es en vez de decirle a CV cuantos SPLITS quiero, 5 o 10, le vamos a pasar CROSS_VALIADATE GENERATOR, es decir
# un generador de SPLITS con KFOLD --->

from sklearn.model_selection import KFold

SEED = 234
np.random.seed(SEED)

#Definimos CV como KFOLD y recibe numero de Splits de 10:
cv = KFold(n_splits = 10)

modelo = DecisionTreeClassifier(max_depth=3)
resultados = cross_validate(modelo, x, y, cv = cv) # A CV le pasamos CV(Kfold)
resultados["test_score"]

media = resultados["test_score"].mean()
media

desviacion_standard = resultados["test_score"].std()
print("Nuestro Intervalo de Confianza para CV = 3 es [%.2f, %.2f]" % ((media - 2*desviacion_standard)*100, (media + 2*desviacion_standard)*100))

# Nos devuelve casi el mismo resultado que sin KFOLD.(Sin KFOLD=Nuestro Intervalo de Confianza para CV = 10 es [77.15, 80.23])


Nuestro Intervalo de Confianza para CV = 3 es [76.58, 80.76]


In [ ]:
# 2°) Ahora lo que vamos a hacer es decirle que quiero 10 PARTES pero DESORDENAME los datos antes, cambiamos el KFOLD para
# recibir SHUFFLE igual a TRUE. Pero antes definimos una FUNCION que imprima los resultados, asi evitamos escribir algunas
# lineas de codigo ---->

def imprime_resultados(resultados):
  media = resultados["test_score"].mean()
  desviacion_standard = resultados["test_score"].std()

  print("Nuestra MEDIA es %.2f" %(media*100))
  print("Nuestro Intervalo de Confianza es [%.2f, %.2f]" % ((media - 2*desviacion_standard)*100, (media + 2*desviacion_standard)*100))


# Usamos codigo de anterior de KFold=10, y aplicamos la Funcion:
SEED = 234
np.random.seed(SEED)

cv = KFold(n_splits = 10)

modelo = DecisionTreeClassifier(max_depth=3)

resultados = cross_validate(modelo, x, y, cv = cv) # A CV le pasamos CV(Kfold)
imprime_resultados(resultados)


Nuestra MEDIA es 78.67
Nuestro Intervalo de Confianza es [76.58, 80.76]


In [ ]:
# 3°) Ahora DESORDENAMOS los datos con SHUFFLE --->

SEED = 234
np.random.seed(SEED)

cv = KFold(n_splits = 10, shuffle=True) # Utilizadmos SHUFFLE paa desordenar datos.

modelo = DecisionTreeClassifier(max_depth=3)

resultados = cross_validate(modelo, x, y, cv = cv) # A CV le pasamos CV(Kfold)
imprime_resultados(resultados)

# Nos devuelve una MEDIA bastante parecida, NO cambio mucho. Pero noestro INTERVALO de CONFIANZA cambio bastante, nos da un
# mejor resultados los datos Desordenados Aleatoriamente.

Nuestra MEDIA es 78.69
Nuestro Intervalo de Confianza es [75.28, 82.10]


**Comentario:**

* Queremos que ellos de verdad sean desordenados antes de ser utilizados porque a veces puede ser que nuestra base, de la manera que la leemos, puede ser que tenga un orden que no sea exactamente el orden que quiero. Imagínense.
Mis datos están ordenados por data y no me gustaría que cuando él hace cross-validation siempre pegue las mismas datas juntas o considere ese factor temporal para ese split. Entonces ese shuffle va a permitirnos exactamente eso, va a permitirnos que no, quiero que primero aleatorice un poco mi base y después consigas hacer el split.
Bueno, solo para concluir, aquí en este test ¿qué hicimos? Utilizamos K-fold, pero en realidad scikit-learn tiene una serie de métodos que podemos usar para explicar y dividir nuestra base y podamos verlo aquí en nuestra página de scikit-learn en el model selection. Aquí vemos todas estas splitter classes.
Entonces aquí por ejemplo nosotros estamos K-fold y hay algunos aquí, como ya mencioné, stratifier K-fold, pueden ser usados para dividir nuestra base y ser usados en nuestro cross-validate.

* Utilizar el método KFold para desordenar (shuffle) los datos.


## **ESTRATIFICACION**

---

### **<font color=red> ESTRATIFICACION CON VALIDACION CRUZADA**

Puede pasar que nuestros SPLITS seleccionen algunos datos que NO estarian deisponibles en la base de evaluacion que podriamos estar usando.

In [ ]:
# 1°) Vamos a simular datos ORDENADOS, vamos a ordenar nuestros datos por nuestro target, para forzar que tengamos mas datos
#de una clase en nuestros Samples mas que en otros Samples --->

datos_ordenados = datos.sort_values(by="vendido") # Ordenamos los valores, que mi base aparezca ORDENADA por vendido.

# Luego definimos de nuevos nuestra X, Y en base a este nuevo ordenamiento de nuestras base:
x_ordenados = datos_ordenados[["precio", "edad_del_modelo", "km_por_ano"]]
y_ordenados = datos_ordenados["vendido"]

datos_ordenados

# Nos devuelve los datos ordenados por valor de menor a mayor por clase VENDIDO.


,precio,vendido,edad_del_modelo,km_por_ano
4999,74023.29,0,12,24812.80412
5322,84843.49,0,13,23095.63834
5319,83100.27,0,19,36240.72746
5316,87932.13,0,16,32249.56426
5315,77937.01,0,15,28414.50704
...,...,...,...,...
5491,71910.43,1,9,25778.40812
1873,30456.53,1,6,15468.97608
1874,69342.41,1,11,16909.33538
5499,70520.39,1,16,19622.68262


In [ ]:
# 2°) Ahora le decimos que ejecute el KFOLD sin SHUFFLE y le pasamos nuestra x_ordenados e y_ordenados --->


cv = KFold(n_splits = 10)

modelo = DecisionTreeClassifier(max_depth=3)

resultados = cross_validate(modelo, x_ordenados, y_ordenados, cv = cv) # A CV le pasamos CV(Kfold)
imprime_resultados(resultados)


# Nos devuelve valores feos, los anteriores sin ordenar los datos por clase y con el SHUFFLE eran mejor, ya que tenina los datos
# DESORDENADOS.

Nuestra MEDIA es 78.67
Nuestro Intervalo de Confianza es [74.13, 83.21]


In [ ]:
# °) Ahora usamos mismos los mismos codigos y le pasamos SHUFFLE igual a TRUE --->


cv = KFold(n_splits = 10, shuffle=True) # Utilizadmos SHUFFLE paa desordenar datos.

modelo = DecisionTreeClassifier(max_depth=3)

resultados = cross_validate(modelo, x_ordenados, y_ordenados, cv = cv) # A CV le pasamos CV(Kfold)
imprime_resultados(resultados)

# Nos devuelve mejores valores.

Nuestra MEDIA es 78.68
Nuestro Intervalo de Confianza es [76.71, 80.65]


In [ ]:
# 4°) Un SPLITTER que nos podria ayudar bastante a mantener en cada PARTE de la devision que estamos haciendo, la misma proporcion
# de nuestra clases seria STRATIFIEDKFOLD ya que KFOLD NO recibe la misma proporcion de los datos --->

from sklearn.model_selection import StratifiedKFold


cv = StratifiedKFold(n_splits = 10, shuffle=True)

modelo = DecisionTreeClassifier(max_depth=3)

resultados = cross_validate(modelo, x_ordenados, y_ordenados, cv = cv) # A CV le pasamos CV(Kfold)
imprime_resultados(resultados)

# MEJORAN AUN MAS LOS VALORES DEL INTERVALO DE CONFIANZA CON STRATIFIEDFOLD

Nuestra MEDIA es 78.68
Nuestro Intervalo de Confianza es [76.08, 81.28]


### **<font color=red> DOCUMENTACION Y BUENAS PRACTICAS**


En <font color=orange>VALIDACION DE MODELOS</font> , vimos varias opciones que tenemos para conseguir evaluar o validar si nuestro modelo de hecho está performando como queríamos.

Por ejemplo:

* Mencionamos de separar nuestra base en entrenamiento y evaluación, vimos que ese abordaje puede tener algunos problemas, porque a veces dependiendo de esa distribución puede ser mejor que dividamos en varias partes y evaluemos varias veces que será nuestro <font color=orange>CROSS-VALIDATION</font>.

* Hemos visto cómo podemos hacer ese <font color=orange>CROSS-VALIDATION</font> más eficiente, si podríamos desordenar los datos o no desordenarlos, tal vez usar un <font color=orange>STRATIFIER KFOLD</font> cuando nuestro target o valor de respuesta no está 50% igualmente distribuida en nuestra base.

* Las fuentes que generalmente usamos para buscar y decidir cuál sería la mejor metodología que usar. que en la propia página de <font color=orange>SICKIT LEARN</font>, la documentación de ellos es muy buena, tiene unos diferentes tipos de clase que podríamos usar y cada una de ellas  dice cuál sería el mejor caso de uso para poder emplearlo.

* Hemos visto que <font color=orange>STRATIFIER KFOLD</font>, como ya hemos visto antes, es bastante bueno cuando queremos preservar el porcentaje de cada clase,<font color=orange>KFOLD</font> sería diferente en otro tipo de situaciones. Podemos ver que ya existe una serie de documentación y de guías para uno ayudarse cada vez que uno tiene alguna duda sobre cómo hacer la mejor selección para nuestro problema.

* Vemos que en la documentacion de <font color=orange>SCIKIT-LEARN</font> hay algunas investigaciones, ya mapeadas con papers y cómo en la ciencia estamos viendo que esos test que tenemos que hacer de cómo hacer que nuestro algoritmo de hecho nos dé las métricas más realistas posibles cuando estamos resolviendo nuestro problema.

* Por ejemplo en esta parte de <font color=orange>SICKIT LEARN</font>, en este módulo de <font color=orange>CROSS-VALIDATION</font> que habla bastante de esa manera de cómo evaluar modelos, podemos ver que tópicos como cuántas veces debería dividir mi base, si debería ser 10 o en 5 folds, ¿cuál será el mejor parámetro?

* Hay algunos papers o algunos libros que hablan sobre cuál sería el mejor parámetro el que deberíamos escoger o sobre lo que comentamos de ser un target desbalanceado, que en este caso <font color=orange>STRATIFIER KFOLD</font> sería la mejor opción.

* Hay bastante documentación y nos puede mandar para otros links externos que tienen otras bases y otros fundamentos técnicos que podrían ayudarlos, entonces no duden en venir y leer la documentación que puede ayudarlos mucho a resolver el problema que tengan.

## **DATOS AGRUPABLES**

---

### **<font color=red> CREANDO DATOS ALEATORIOS**

Ya vimos anteriormente como podriamos usar STRATIFIEDKFOLD o solamente KFOLD para luego usar CROSS-VALIDATE para nuestros datos, pueden estar ordenados o desordenados haciendo SHUFFLE.

In [ ]:
# 1°) Vamos a ver como hacemos para hacer la evaluacion de los GRUPOS.

# Por ejemplo: supongamos un modelo para un hospital y en ella tenemos una base de Clientes, y algunos de esos clientes son nuevos
# y cuando colocamos ese modelo en produccion lo que queremos es que esa produccion funcione tanto para los clientes que ya fueron
# analizados como tambien para los clientes nuevos y para eso tendriamos que tener 2 variables para que algunos clientes que ya fueron
# analizados NO esten en la base de TEST(evaluacion).

# En nuestra base de datos por ejemplo EDAD_DEL_MODELO seria una variable que podriamos segmentar por GRUPOS, ya que en un año hay modelos
# que estan en circulacion pero tambien en otros años van a venir otros modelos totalmente diferentes y el modelo debe conseguir verificar eso.
# STRATIFIEDKFOLD, NO funcionaria en este caso ya que NO consideraria cuando haga el SPLIT aleatoreo de la base, el considera que son iguales,
# para esto tenemos que hacer antes una variable de GRUPOS, para esto craemos la variable MODELO de auto --->

datos.edad_del_modelo.head() # vemos edad del modelo


0    18
1    20
2    12
3     3
4     4
Name: edad_del_modelo, dtype: int64

In [ ]:
# Importamos NUMPY y creamos la columna de MODELO de auto:

import numpy as np # Usamos numpy para crear numeros aleatorios a edad_del_modelo

SEED=1234 # Para que nos devuelva mismo valores
np.random.seed(SEED)

np.random.randint(-2, 3, size = len(datos))# LOW(-2)=menor n° que queremos utilizar, HIGH(3)=mayor n° que queremos utilizar, SIZE=Crear n° Aleatorios para valores de datos

# Nos devuelve un array

array([ 1,  2,  2, ..., -2,  2, -2])

In [ ]:
# 2°) Ahora creamos la columna de numeros aleatorios MODELO y le sumamos los numeros aleatorios a los datos de EDAD_DEL_MODELO ---->

SEED=1234 # Para que nos devuelva mismo valores
np.random.seed(SEED)

datos["modelo"] = datos.edad_del_modelo + np.random.randint(-2, 3, size = len(datos))
datos.head()

,precio,vendido,edad_del_modelo,km_por_ano,modelo
0,30941.02,1,18,35085.22134,19
1,40557.96,1,20,12622.05362,22
2,89627.50,0,12,11440.79806,14
3,95276.14,0,3,43167.32682,1
4,117384.68,1,4,12770.11290,3


In [ ]:
# 3°) Ahora vemos los valores de MODELO --->

datos.modelo.unique()

# Nos devuelve los numeros aleatorios de MODELO asignados para EDAD_DEL_MODELO, nos muestra que hay un valor -1 y no es coveniente tener
# valores NEGATIVOS en la base de datos, para revertir esto le aplicamos VALOR ABSOLUTO.

array([19, 22, 14,  1,  3, 10, 15, 20, 17,  4,  9, 11, 18,  6, 16,  5,  8,
       12, 21, 13,  7,  2,  0, -1])

In [ ]:
# 4°) Le agregamos VALOR ABSOLUTO a los datos del MODELO para convertir el -1 en 0 y sumarle 1 al numero Aleatorio --->

# Para ver el minimo que es el -1
min(datos.modelo)

datos.modelo + abs(min(datos.modelo)) +1

0       21
1       24
2       16
3        3
4        5
        ..
9995    16
9996    16
9997     4
9998    11
9999    19
Name: modelo, Length: 10000, dtype: int64

In [ ]:
# 5°) Agragamos el valor absoluto a variable creada mas arriba --->


SEED=1234 # Para que nos devuelva mismo valores
np.random.seed(SEED)

datos["modelo"] = datos.edad_del_modelo + np.random.randint(-2, 3, size = len(datos))
datos["modelo"] = datos.modelo + abs(min(datos.modelo)) +1
datos.head()

,precio,vendido,edad_del_modelo,km_por_ano,modelo
0,30941.02,1,18,35085.22134,21
1,40557.96,1,20,12622.05362,24
2,89627.50,0,12,11440.79806,16
3,95276.14,0,3,43167.32682,3
4,117384.68,1,4,12770.11290,5


In [ ]:
# Comprobamos:

datos.modelo.unique()

array([21, 24, 16,  3,  5, 12, 17, 22, 19,  6, 11, 13, 20,  8, 18,  7, 10,
       14, 23, 15,  9,  4,  2,  1])

Definicion Aula:

* GroupKFold agrupa por grupo. Cuando utilizamos GroupKFold tenemos que dejar definidos los grupos que nos gustaría agrupar y la columna del grupo.

### **<font color=red> VALIDACION CRUZADA USANDO GRUPOS**

In [ ]:
# 1°) Ya tenemos el MODELO del auto en la columna, podemos comenzar a evaluar el entrenamiento, en este caso vamos a usar
# GROUPKFOLD, lo que hace es cada grupo nos va a aparecer en 2 diferentes FOLDS, es decir en 2 partes diferentes de mis datos,
# NO van a ser grupos iguales --->

from sklearn.model_selection import GroupKFold

cv = GroupKFold(n_splits=10)

modelo = DecisionTreeClassifier(max_depth = 2)

resultados = cross_validate(modelo, x, y, cv=cv, groups=datos.modelo) # le pasamos datos de la columna MODELO
imprime_resultados(resultados)

# La media continua igual.
# El intervalo de confianza esta un poco diferente, se amplio un poco mas! En comparacion con STRATIFIEDKFOLD que no tenemos grupos definidos.

Nuestra MEDIA es 75.78
Nuestro Intervalo de Confianza es [70.82, 80.74]


## **PIPELINE DE ENTRENAMIENTO Y VALIDACION**

---

### **<font color=red> IMPORTANCIA DE UN PIPELINE EN CROSS-VALIDATION**

* Vimos cómo usar el **GroupKFold** cuando alguna de nuestras variables es importante para segmentar en los grupos que queremos hacer entrenamiento y evaluación y no tener todos los grupos dentro de todos los splits que vamos a tener de cross-validation.
Entonces en este caso simulamos unos datos de algunos modelos de autos y los colocamos aquí dentro de nuestra categoría de grupos para que **cross-validate** pudiera hacer eso, colocar en algunos de nuestros splits algunos grupos de autos y en otros no. Pudimos ver durante el curso usamos decision tree.

* Una curiosidad de **árboles de decisión** es que él no está afectado por la magnitud de las variables que colocamos, entonces si por ejemplo si mis variables que en nuestro caso están en escala de centenas o en escala de miles, para **decision tree** no hace ninguna diferencia. No tiene ninguna diferencia si mis escalas están diferentes.

* Pero dependiendo del modelo que escogemos, puede ser que esto sea un factor que debamos considerar. Entonces por ejemplo en modelos lineales, generalmente la magnitud de las variables afecta bastante cómo nuestro modelo performa al final, cómo el modelo va a ser ejecutado.

* Entonces en vez de hacer todos los cross-validation como lo estamos haciendo con decision tree, hiciéramos un cross- validation con una **REGRESION LOGISTICA** donde necesitamos hacer ese pre procesamiento de nuestras variables explicativas para que nuestro modelo pueda performar muy bien.




In [ ]:
# 1°) Vamos primero a ver cómo se comportaría nuestro cross-validation si en vez de usar decision tree classifier usase una
# REGRESION LOGISTICA --->

from sklearn.linear_model import LogisticRegression

cv = GroupKFold(n_splits=10)

modelo = LogisticRegression() # Dejamos parametros por defecto

resultados = cross_validate(modelo, x, y, cv=cv, groups=datos.modelo) # le pasamos datos de la columna MODELO
imprime_resultados(resultados)

# Funciona peor de lo que estabamos acostumbrados a obtener con DecisionTreeClassifier


Nuestra MEDIA es 60.61
Nuestro Intervalo de Confianza es [55.84, 65.38]


In [ ]:
# 2°) Como LOGISTIC REGRESSION es una REGRESION LINEAL y se ve afectada por la escala de nuestras variables, vamos a modificar las escalas
# de las variables para que esten estandarizadas y usaremos STANDARD SCALER ---->

from sklearn.preprocessing import StandardScaler

scaler = StandardScaler() # Instanciamos

scaler.fit(train_x) # Aprendemos con los valores ya entrenados que fueron elaborados mas arriba(train_x)

train_x_scaled = scaler.transform(train_x) # Transformamos base de entrenamiento y evaluacion con Scaler.
test_x_scaled = scaler.transform(test_x)

train_x_scaled # Comprobamos que estandarizo los valores

# Vemos que todas las variables estan en la misma escala estandarizada.

array([[ 1.62663622,  0.66907903, -0.62913537],
       [-0.33714655,  0.24235232, -1.31870128],
       [ 0.76029211,  0.66907903,  0.65698987],
       ...,
       [ 1.5520527 ,  0.45571567,  1.5672967 ],
       [ 0.44233999,  0.02898897, -0.99218034],
       [-0.0212041 ,  1.09580574, -0.67583824]])

In [ ]:
# 3°) Vamos a rodar solo el modelo con REGRESION LOGISTICA sin CROSS VALIDATION --->

SEED = 158020
np.random.seed(SEED)

modelo = LogisticRegression()
modelo.fit(train_x, train_y)

predicciones = modelo.predict(test_x_scaled) # pasamos en predicciones test x scaled con los valores estandarizadosSEED = 158020

accuracy = accuracy_score(test_y, predicciones) * 100
print("La exactitud(accuracy) fue %.2f%%" % accuracy)

# Nuestro SCALER nos ayudo para que nos diera un poco mejor.

La exactitud(accuracy) fue 54.12%


/usr/local/lib/python3.10/dist-packages/sklearn/base.py:439: UserWarning: X does not have valid feature names, but LogisticRegression was fitted with feature names
  warnings.warn(


In [ ]:
# 4°) Ahora hacemos un CROSS VALIDATION que separa y entrena varias veces mi modelo. Y para esto haremos un PIPELINE que recibe
# diferentes pasos que necesito para llegar a mi mocdlo final, pasandole GROUPKFOLD, CROSS VALIDATION y LOGISTIC REGRESSION --->

from sklearn.pipeline import Pipeline

SEED = 158020
np.random.seed(SEED)

scaler = StandardScaler() # Definimos que scaler es igual a StandardScaler
modelo = LogisticRegression() # Definimos que modelo es igual a LogisticRegression
pipeline = Pipeline([("transform", scaler), ("model", modelo)]) # Definimos que pipeline es igual a Pipeline y le pasamos parametros.

cv = GroupKFold(n_splits=10)

resultados = cross_validate(pipeline, x, y, cv=cv, groups=datos.modelo )# Pasamos PIPELINE en lugar de modelo, ya que tiene todos los pasos de Scaler,
# modelo y Pipeline.
imprime_resultados(resultados)

# Mejora mucho nuestros resultados ESCALANDO los valores de las variables, antes de usar SCALER en Regresion Logistica los resultados fueron valores
# menores, luego de usar SCALER en Regresion Logistica mejoro mucho nuestro Accuracy. CROSS_VALIDATION tiene esa particularidad de poder hacer un
# PIPELINE y pasar todos los pasos que queremos hacer para construir el modelo final posible.

Nuestra MEDIA es 70.39
Nuestro Intervalo de Confianza es [65.39, 75.38]


<font color=orange>**Definicion**:

*Mario es un analista de datos que está trabajando en una empresa financiera. Él está haciendo un preprocesamiento y evaluación de datos en su empresa, pero este proceso está siendo ejecutado sólo una vez.
A Mario le gustaría ejecutar ese proceso varias veces con diferentes grupos, ¿cómo podría hacer?*

* Utilizando una pipeline para hacer el preprocesamiento varias veces.
Con una pipeline podemos ejecutar el preprocesamiento varias veces.

<font color=orange>**Conclusion de lo que hicimos:**

* Vamos a recordar todo lo que hicimos durante este curso. Inicialmente comenzamos nuestra clase leyendo una base de datos donde cada línea representaba un carro en una tienda específica, con algunas características, y sabíamos cuáles de ellos habían sido vendidos o no, que era lo que queríamos especificar.
Queríamos en realidad un modelo de clasificación que me diga, dado algunas características de ese carro, cuál es la probabilidad de esa carro de hecho ser vendido o no. Entonces, como no sabíamos mucho, comenzamos primero a separar nuestra base en entrenamiento y evaluación, y decidimos crear un domain classifier solo para tener una idea de cómo nuestra base está distribuida en relación a la target, y un clasificador bastante simple podría resolver.

* Y vimos que en este caso nuestro accuracy fue bastante bajo, fue de 51%, lo que no es un buen modelo en general. Y ahí pensamos, tal vez un domain classifier no sería la mejor opción. Lo que deberíamos hacer entonces tal vez es solo separar y crear un modelo de clasificación un poco más robusta que podría ser decision tree. En este caso hicimos un decision tree bastante simple de máximo de profundidad 2 y vimos que ya con eso pudimos mejorar nuestro accuracy para 72%.

* Pero si se acuerdan, vimos que dependiendo de ese seed que colocábamos de quiénes serían nuestra división de entrenamiento y validación, nosotros comenzamos a ver que en realidad esa predicción podría estar un poco afectada de qué datos le paso para que aprenda y qué datos uso para evaluar.
Entonces vimos que una posibilidad que podríamos usar para evaluar nuestros modelos podría ser separar nuestra base varias veces en diferentes grupos de entrenamiento y otros grupos de evaluación y conseguir calcular nuestra performance o nuestra métrica esperada, que podría ser una media, y al mismo tiempo también calcular cuál podría ser ese intervalo de confianza que tendríamos.

* Entonces ya no tendríamos una métrica puntual, un valor de nuestra métrica, sino podríamos tener ese intervalo donde yo espero que mi métrica performe cuando coloco ese modelo de una vez en producción. Ahí después vimos que ese cross-validation estaba afectado en este caso por en cuántas partes quiero dividir mi base de datos.
Entonces cuando intentamos con 3, intentamos con 5, con 10, vimos que nuestros resultados eran un poco diferentes, y ahí después descubrimos que en realidad algunas buenas prácticas que ya inclusive scikit-learn tiene documentadas y algunos papers ya tienen disponible, es que a partir de 5 ya nuestro modelo podríamos confiar en ese resultado que estamos obteniendo.

* Luego pasamos aquí un poco de cómo podría tener mala suerte de que si nuestros datos están ordenados en una cierta forma, eso podría afectar cómo mi cross-validate de hecho está separando mi base en esas pequeñas muestras de entrenamiento y de evaluación.
Entonces hemos generado esos datos ordenados para simular esa mala suerte y vimos que nuestro KFold de hecho podría estar afectado. Pero tenemos una solución que sería incluir este shuffle en mi KFold, que lo que va a hacer es desordenar un poco mis datos para que el orden en el que le estoy pasando esas informaciones no influya en cómo el KFold o nuestro cross-validation va a de hecho seleccionar nuestras bases.

* Después vimos que tenemos otras opciones en KFold. Nuestra base es seleccionada aleatoriamente pero no necesariamente mantiene la proporción de nuestro target inicial. Entonces vamos a suponer, si tendríamos más representantes de nuestra clase 1, podría ser que algunos samples no queden con la misma proporción, y nos gustaría forzar un poquito de esos.
Entonces vimos que stratifier KFold es una opción bastante utilizada cuando queremos eso, queremos que cada sample que estamos generando, de hecho esté con la misma proporción de nuestra variable respuesta.

* Después vimos que ese stratifier KFold también tiene la misma opción de shuffle para desordenar nuestros datos y así podríamos inclusive no tener ese problema del orden que le estamos pasando de nuestra base. Después finalmente vimos que en algunos casos me gustaría entender cómo mi modelo funciona cuando le paso datos nuevos que n o estarían disponibles cuando estoy entrenando.
Entonces por ejemplo en el caso de nuestros carros simulamos y creamos una columna nueva que sería el modelo del carro, que podría ser esta variable que me diría: "Bueno, algunos carros no estarían disponibles siempre cuando voy a entrenar y me gustaría que nuestro modelo evaluase en carros diferentes nuevos para simular esa situación de entrar nuevos carros en el mercado y así".

* Entonces simulamos nuestra columna aquí con nuestra base ficticia, decidimos simular esa columna nueva de modelo del carro y después rodamos de nuevo nuestro cross-validation, pero ahorita incluyendo un group KFold que incluye esa variable de grupo que estábamos interesados que fuese incluida.
Finalmente vimos que en el caso de que tenemos diferentes pasos para calcular nuestro predictor, por ejemplo en uno de los lineares que tenemos que hacer unas transformaciones previas para nuestras variables, vimos que pipeline es una excelente opción cuando queremos colocar esos pasos para poder de hecho rodar y ejecutar nuestro KFold, al final nuestro cross-validation.
Entonces en este caso lo rodamos y vimos que es posible inclusive incluir todos esos pasos que tenemos que hacer para que nuestro entrenamiento de nuestro modelo sea el adecuado.



